In [32]:
import numpy as np
from collections import defaultdict, Counter, OrderedDict
import featureselection as fs
import os.path
import os
import nltk
from nltk.corpus import stopwords
import datetime
from operator import itemgetter
import preprocessing as pp
import time

In [29]:
def softmax(x):
    b = x.max()
    y = np.exp(x - b)
    return y / y.sum()

In [3]:
def sigmoid(x):
    return .5 * (1 + np.tanh(.5 * x))

In [33]:
def logistic_regression(weights, data, binary_label_vectors, threshold, learning_rate):
    '''
    :param weights: a matrix of len(valid_words) rows and len(label_list)
                    columns, where entry i,j is the weight for word i
                    in documents with label j
    :param data: a matrix of len(valid_words) rows and num_docs columns,
                where entry i,j is the number of occurrences of word i
                in document j
    :param binary_label_vectors: a matrix of len(label_list) rows and
                                num_docs cols, where each entry i,j is
                                1 if document j has label i, and 0 otherwise
    :param threshold: the value at which if all weights only change by less than
                        this value, we claim the model has converged
    :param learning rate: the scaling factor of the gradient to ensure we do not
                            descend too quickly
    :return: the weights matrix, updated after running gradient descent on it 
    '''
    for label in binary_label_vectors.keys():
        print("Training label", label)
        labels = np.asarray(binary_label_vectors[label]).reshape(1, data.shape[1])
        # Labels.shape = 1x5
        num_docs = labels.shape[0]
        weight_vector = np.transpose(weights[label]).reshape(1, data.shape[0])
        converged = False
        k = 1
        num_docs = labels.shape[0]
        while not converged and k < 50:
            converged = True
            print(label, "Iteration:", k)
            start = time.time()
            weight_eval = sigmoid(np.dot(weight_vector, data))
            # weight_eval.shape = 1x5
            print("W^T * x", time.time() - start)
            start = time.time()
            loss = np.subtract(labels, weight_eval)
            # loss.shape = 1x5
            gradient = np.dot(loss, np.transpose(data))
            gradient *= learning_rate
            print("Gradient: ", time.time()-start)
            start = time.time()
            updated_weights = weight_vector + gradient
            delta = weight_vector - updated_weights
            weight_vector = updated_weights
            for i in range(len(delta)):
                for j in range(len(delta[0])):
                    if np.absolute(delta[i][j]) >= threshold:
                        converged = False
            k += 1
            print("Check convergence: ", time.time()-start)
            # gradient.shape = 1x308
        # print("Iterations: ", label, k)
        weights[label] = weight_vector
    return weights

In [34]:
def build_new_cats_file(filename):
    '''
    This is a utility function that will rearrange the elements
    in the cats.txt file so that they are sorted by numbers instead
    '''
    with open(filename, "r") as f:
        training_lines = []
        test_lines = []
        for line in f:
            elems = line.split()
            if line[0:8] == "training":
                num = int(elems[0][9:len(elems[0])])
                training_lines.append((num, elems[1:]))
            else:
                num = int(elems[0][5:len(elems[0])])
                test_lines.append((num, elems[1:]))
    training_lines = sorted(training_lines, key=lambda x: x[0])
    test_lines = sorted(test_lines, key=lambda x: x[0])
    with open("cats2.txt", "w") as f:
        for num, labels in training_lines:
            f.write("training/" + str(num) + " " + " ".join(labels) + "\n")
        for num, labels in test_lines:
            f.write("test/" + str(num) + " " + " ".join(labels) + "\n")

In [35]:
def construct_feature_matrix(dir_path, valid_words):
    '''
    This function will take text vectors of each document
    and put them into a matrix.
    :param dir_path: path to the directory containing all the
                    training files
    :param valid_words: a dictionary where keys are all the 
                        valid words in the corpus
    '''
    feature_matrix = []
    sorted_files = sorted([int(file[0:len(file) - 4])  for file in os.listdir(dir_path)])
    for num in sorted_files:
        filepath = dir_path + "\\" + str(num) + ".txt"
        text_vector = pp.vectorize_text(valid_words, filepath)
        word_freq = {word: 0 for word in valid_words}
        freq = Counter(text_vector)
        for word in freq.keys():
            word_freq[word] = freq[word]
        frequencies = [y for x,y in word_freq.items()]
        feature_matrix.append(frequencies)
    return np.transpose(np.asarray(feature_matrix))

In [36]:
def build_text_vectors(dir_path, valid_words):
    text_vectors = defaultdict()
    sorted_files = sorted([int(file[0:len(file) - 4])  for file in os.listdir(dir_path)])
    for num in sorted_files:
        frequencies = {word: 0 for word in valid_words}
        filepath = dir_path + "\\" + str(num) + ".txt"
        text_vector = pp.vectorize_text(valid_words, filepath)
        freq = Counter(text_vector)
        for word in freq.keys():
            frequencies[word] = freq[word]
        text_vectors[num] = np.asarray([y for x,y in frequencies.items()])
    return text_vectors

In [37]:
def construct_binary_label_vectors(indexed_labels, filename, num_docs):
    '''
    This function will return a dictionary where keys = labels 
    and values = a 2D vector of len(label_list) rows and len(num_docs)
    cols where each entry i,j is 1 if word i has label j and 0 otherwise
    :param label_list: dict where keys = labels and values = index
                        that maps to that label
    :param filename: name of the file that maps doc numbers to 
                    labels
    '''
    binary_label_vectors = {label: [0.0 for i in range(num_docs)] for label in indexed_labels.keys()}
    j = 0
    with open(filename, "r") as f:
        for line in f:
            terms = line.split()
            if terms[0][0:4] == "test":
                continue
            for elem in terms[1:]:
                binary_label_vectors[elem][j] = 1.0
            j += 1
    return binary_label_vectors                              

In [11]:
if __name__ == '__main__':
    dir_path = "C:\\Users\\ksing\\OneDrive\\Documents\\TextClassifiers\\training"
    stop_words = set(stopwords.words('english'))
    valid_words = fs.most_useful_features("cdmscores.txt")
    # valid_words = pp.get_valid_words(dir_path, stop_words)
    number_labels_training, number_labels_test = pp.add_labels_to_samples("cats2.txt")
    prior_probs = pp.compute_prior_probabilities(number_labels_training)
    indexed_labels = {label: i for i, label in enumerate(prior_probs.keys())}

14235


In [12]:
    # build_new_cats_file("cats.txt")
    feature_matrix = construct_feature_matrix(dir_path, valid_words)
    print(feature_matrix.shape)
    # Each column in the feature matrix corresponds to an individual document

(14235, 7769)


In [13]:
    weights = {label: np.asarray([np.random.uniform(0,0.005) for i in range(len(valid_words.keys()))]) for label in prior_probs.keys()}

In [14]:
    num_docs = len([name for name in os.listdir(dir_path) if os.path.isfile(os.path.join(dir_path, name))])
    binary_label_vectors = construct_binary_label_vectors(indexed_labels, "cats2.txt", num_docs)

In [38]:
    trained_weights = logistic_regression(weights, feature_matrix, binary_label_vectors, 0.01, 0.01)

Training label cocoa
cocoa Iteration: 1
W^T * x 0.5385594367980957
Gradient:  0.5160431861877441
Check convergence:  0.02094721794128418
cocoa Iteration: 2
W^T * x 0.46951985359191895
Gradient:  0.46779704093933105
Check convergence:  0.03889656066894531
cocoa Iteration: 3
W^T * x 0.4388282299041748
Gradient:  0.44880008697509766
Check convergence:  0.022938013076782227
cocoa Iteration: 4
W^T * x 0.4517941474914551
Gradient:  0.46076536178588867
Check convergence:  0.024933576583862305
cocoa Iteration: 5
W^T * x 0.4448108673095703
Gradient:  0.4946780204772949
Check convergence:  0.021941661834716797
cocoa Iteration: 6
W^T * x 0.463759183883667
Gradient:  0.4946770668029785
Check convergence:  0.021941423416137695
cocoa Iteration: 7
W^T * x 0.43184804916381836
Gradient:  0.4507942199707031
Check convergence:  0.021939754486083984
cocoa Iteration: 8
W^T * x 0.4388270378112793


KeyboardInterrupt: 

In [31]:
    successes = 0
    j = 0
    dir_path = "C:\\Users\\ksing\\OneDrive\\Documents\\TextClassifiers\\test"
    for file in os.listdir(dir_path):
        num = int(file[0:len(file) - 4])
        filepath = dir_path + "\\" + file
        vector = pp.vectorize_text(valid_words, filepath)
        frequencies = {word: 0 for word in valid_words}
        freq = Counter(vector)
        for word in freq.keys():
            frequencies[word] = freq[word]
        counts = np.asarray([v for v in frequencies.values()])
        probs = []
        for label, w in trained_weights.items():
            prob = np.dot(w, counts)
            probs.append([label, prob[0]])
        probs = sorted(probs, key=lambda t: t[1], reverse=True)
        just_scores = np.array([y for x,y in probs])
        just_scores = softmax(just_scores)
        for i in range(len(just_scores)):
            probs[i][1] = just_scores[i]
        s,e,b = pp.accuracy_model(num, number_labels_test, probs)
        successes += s
        j += 1
        # First run, 86.07% accuracy
    print(successes, j)
            

14832
['rubber', 'tin', 'sugar', 'corn', 'rice', 'grain', 'trade'] [['trade', 0.9831750156912388], ['gnp', 0.016824984308761153], ['crude', 2.67231661697633e-24], ['bop', 1.5362238847078884e-27], ['earn', 7.923225369668242e-33], ['interest', 1.521979411901431e-46], ['corn', 4.8574772299958826e-67], ['grain', 8.950971604636111e-68], ['money-fx', 3.805498842655175e-74], ['nat-gas', 1.2252263333016753e-84]]
14840
['rubber', 'coffee', 'lumber', 'palm-oil', 'veg-oil'] [['coffee', 1.0], ['money-fx', 4.428814953463772e-47], ['ship', 1.014455720764093e-64], ['trade', 1.0200815236244184e-68], ['acq', 1.79369362465639e-75], ['grain', 2.934829382042259e-78], ['crude', 2.3228075870005142e-90], ['earn', 1.6272171826129075e-115], ['sugar', 4.217212589005617e-124], ['wheat', 1.7310939563603194e-126]]
14844
['tin'] [['acq', 0.9999999415228805], ['ship', 5.78693018702331e-08], ['money-fx', 6.074576858705306e-10], ['crude', 3.5987669361505883e-13], ['trade', 3.396002716058448e-17], ['interest', 2.736230

15129
['rapeseed', 'oilseed'] [['oilseed', 0.9999982051637564], ['grain', 1.5862309223089513e-06], ['ship', 1.217870327705642e-07], ['wheat', 5.179373571099697e-08], ['corn', 3.502455281120871e-08], ['soybean', 2.6028994190901198e-17], ['sugar', 3.897075609650853e-18], ['iron-steel', 5.1547192018728145e-20], ['trade', 1.454037128626088e-20], ['money-fx', 3.0663338863263566e-21]]
15179
['cocoa'] [['interest', 0.9999999999867422], ['cocoa', 1.325783572477473e-11], ['money-fx', 2.5827836111680215e-32], ['grain', 3.9682752497850504e-46], ['trade', 1.6561539751746203e-51], ['ship', 4.058998312780255e-54], ['acq', 4.782521888926479e-59], ['earn', 2.007687247531063e-62], ['crude', 7.910072181300262e-67], ['wheat', 1.0060126562637048e-76]]
15206
['rice'] [['grain', 0.9999999996740501], ['trade', 2.1635480576352065e-10], ['oilseed', 1.0959497097689604e-10], ['wheat', 2.994648541791863e-16], ['livestock', 1.013255689892149e-29], ['ship', 4.7764826283585575e-30], ['interest', 4.2869471613467873e-

15441
['reserves'] [['cocoa', 0.011111111111111112], ['sorghum', 0.011111111111111112], ['oat', 0.011111111111111112], ['barley', 0.011111111111111112], ['corn', 0.011111111111111112], ['wheat', 0.011111111111111112], ['grain', 0.011111111111111112], ['sunseed', 0.011111111111111112], ['oilseed', 0.011111111111111112], ['soybean', 0.011111111111111112]]
15448
['money-fx'] [['interest', 0.9369040718005077], ['money-fx', 0.06309592819948992], ['grain', 2.2336660909703358e-15], ['earn', 3.9486064853272244e-23], ['acq', 1.101849431484848e-30], ['crude', 7.659225451644765e-32], ['ship', 5.3610225458397754e-39], ['corn', 1.2971977106536308e-39], ['trade', 1.4238860876317432e-41], ['wheat', 8.06784468556755e-46]]
15452
['money-fx', 'trade'] [['money-fx', 1.0], ['gnp', 2.3799154501329715e-20], ['yen', 7.361057294988285e-25], ['interest', 2.7184778823098586e-32], ['trade', 2.1422981292535943e-32], ['dlr', 2.1163587192760964e-32], ['coffee', 5.530342144413132e-44], ['bop', 1.0034097953000829e-45

15639
['iron-steel'] [['earn', 1.0], ['acq', 2.2591592787239554e-65], ['crude', 6.99219596625837e-131], ['trade', 7.807798727499779e-242], ['ship', 2.2546223289936088e-247], ['nat-gas', 4.231395021812823e-272], ['money-fx', 1.2236374119913768e-283], ['grain', 7.021157915203667e-299], ['interest', 3.36e-322], ['gnp', 0.0]]
15643
['cotton'] [['grain', 0.9999999999993763], ['wheat', 6.237218008415771e-13], ['corn', 3.321115258082164e-24], ['oilseed', 2.251634702302057e-24], ['ship', 3.5059529596743214e-28], ['trade', 2.5432029252723444e-31], ['soybean', 8.42019255645944e-33], ['sugar', 5.9237004308175445e-34], ['cotton', 8.004953302675848e-44], ['money-fx', 7.92632114578e-47]]
15649
['barley', 'grain'] [['barley', 1.0], ['iron-steel', 2.5429718137610005e-34], ['veg-oil', 1.5055927484519146e-35], ['coffee', 3.469560380828568e-38], ['grain', 1.5161100368521208e-38], ['carcass', 5.226677795967113e-39], ['livestock', 2.6875577560437978e-39], ['soy-oil', 9.226293418019297e-40], ['meal-feed', 3

15858
['potato'] [['earn', 0.9989607462880237], ['grain', 0.0010392537119762652], ['soybean', 3.000619800578722e-19], ['oilseed', 1.8780975395705672e-23], ['corn', 1.741518658429261e-29], ['sorghum', 3.0385566053975724e-41], ['wheat', 1.1869851043118277e-42], ['sugar', 2.1212753655194166e-53], ['livestock', 5.487222419822816e-56], ['trade', 4.831049475685977e-62]]
15859
['potato'] [['earn', 1.0], ['grain', 2.434058410118851e-18], ['corn', 5.621719325156053e-34], ['sorghum', 3.154251768111448e-36], ['wheat', 6.06946573294136e-40], ['soybean', 2.6438152290290563e-42], ['oilseed', 2.832520512899838e-45], ['barley', 2.6409798909117185e-47], ['cotton', 5.061358921551512e-50], ['livestock', 4.3817665679010785e-52]]
15864
['wheat', 'grain'] [['soybean', 0.9996114140630797], ['sugar', 0.000388584980302269], ['wheat', 9.566170917896733e-10], ['oilseed', 9.351944872809561e-16], ['grain', 1.5676771898684276e-22], ['veg-oil', 2.9681815364025094e-28], ['sorghum', 2.889683612699192e-34], ['corn', 3.

...

19721
['soybean', 'oilseed', 'corn', 'grain'] [['corn', 0.8120992565826255], ['grain', 0.18790074341737462], ['oilseed', 1.3634135412040903e-70], ['ship', 5.185202815885248e-71], ['crude', 1.7185453997620162e-71], ['acq', 5.002121743631317e-72], ['trade', 5.10944261412596e-74], ['money-fx', 1.870754982505904e-77], ['interest', 1.0356104015076466e-82], ['sugar', 7.474336818096518e-92]]
19764
['silver', 'lead', 'zinc', 'gold'] [['gold', 1.0], ['earn', 2.345853802269178e-141], ['acq', 1.380402926717289e-164], ['crude', 2.0880347932149995e-166], ['ship', 4.746870673612873e-189], ['interest', 1.4835182631009482e-191], ['nat-gas', 7.836229363337677e-194], ['money-fx', 6.924066764896205e-198], ['copper', 4.399208306171023e-205], ['trade', 5.37526268300895e-209]]
19828
['acq'] [['earn', 0.9999999999999656], ['acq', 3.4527292561349653e-14], ['crude', 5.96137515874656e-23], ['ship', 1.3517716044296815e-24], ['money-fx', 4.46689745195506e-44], ['trade', 9.213838730668324e-55], ['grain', 9.3741468

20232
['veg-oil'] [['grain', 1.0], ['ship', 3.207292263744317e-36], ['trade', 8.877057756305271e-40], ['wheat', 1.645343636512698e-41], ['crude', 7.420655999407028e-60], ['acq', 1.9207739611103488e-60], ['corn', 1.2258638685525962e-66], ['money-fx', 1.235146009432978e-71], ['sugar', 4.846112132454994e-87], ['interest', 9.470174314880387e-93]]
20248
['jobs', 'trade'] [['cocoa', 0.011111111111111112], ['sorghum', 0.011111111111111112], ['oat', 0.011111111111111112], ['barley', 0.011111111111111112], ['corn', 0.011111111111111112], ['wheat', 0.011111111111111112], ['grain', 0.011111111111111112], ['sunseed', 0.011111111111111112], ['oilseed', 0.011111111111111112], ['soybean', 0.011111111111111112]]
20264
['reserves'] [['trade', 1.0], ['reserves', 1.0429782294944264e-19], ['bop', 1.4452615507393748e-78], ['money-fx', 1.2802175139017137e-89], ['crude', 5.451498789100652e-100], ['earn', 1.7833713137875997e-117], ['grain', 1.3240838313382769e-140], ['acq', 6.111466412459636e-150], ['gnp', 1.

20723
['corn', 'grain', 'soy-meal', 'meal-feed'] [['corn', 1.0], ['grain', 1.0488284738398069e-47], ['acq', 3.639082373245083e-84], ['crude', 1.1487501463148755e-106], ['trade', 7.128903958103292e-117], ['earn', 3.3257807884125776e-121], ['money-fx', 7.422402808081083e-124], ['bop', 2.721794363815725e-125], ['wheat', 3.604658452705115e-126], ['oilseed', 9.993045426307051e-127]]
20773
['earn'] [['acq', 1.0], ['earn', 2.0981507311070746e-83], ['crude', 4.975089572077368e-117], ['grain', 5.9650931762733244e-170], ['money-fx', 2.3349407678801993e-182], ['trade', 1.4575813030168873e-186], ['ship', 3.18939644465841e-188], ['interest', 1.3046595472476677e-205], ['corn', 4.0734649926438455e-210], ['nat-gas', 1.561069792158229e-224]]
20790
['lumber'] [['acq', 1.0], ['ship', 5.68602106438289e-17], ['earn', 3.8345361398610714e-17], ['grain', 2.859982888187719e-17], ['crude', 1.1585745238479939e-17], ['corn', 8.46780030268315e-46], ['trade', 5.659138487247398e-48], ['money-fx', 2.197733971712766e-

21339
['money-fx'] [['interest', 1.0], ['money-fx', 3.1555965806247243e-20], ['acq', 4.640265818983032e-36], ['trade', 2.5504974716676797e-37], ['dlr', 2.6452582506302365e-38], ['ship', 6.599915545940494e-40], ['gnp', 9.545801654733562e-42], ['crude', 1.4323474733647736e-45], ['grain', 1.5383471508585115e-54], ['corn', 3.334922444395706e-66]]
21343
['money-fx'] [['interest', 0.9999999999983007], ['money-fx', 1.699241632136639e-12], ['gnp', 1.4183348723412894e-30], ['crude', 8.035930410648426e-36], ['trade', 2.8875860456844044e-36], ['acq', 2.7751398213166532e-40], ['grain', 4.687970443420629e-45], ['money-supply', 2.2294280664469203e-51], ['ship', 8.500481929587026e-55], ['nat-gas', 7.578170518409052e-58]]
21358
['acq'] [['iron-steel', 1.0], ['acq', 9.932348313711384e-28], ['earn', 2.0933282887251973e-46], ['crude', 4.2717602082956726e-52], ['trade', 1.4425348161266925e-65], ['ship', 6.445315713833789e-69], ['grain', 2.193220750119017e-69], ['nat-gas', 9.398580771553088e-73], ['corn', 

21541
['crude'] [['nat-gas', 0.9999999999999636], ['crude', 3.633529932483136e-14], ['earn', 1.9388163847932175e-109], ['acq', 1.810333680635063e-110], ['ship', 1.147300546518747e-197], ['interest', 8.670388756265871e-218], ['trade', 2.420515176939584e-223], ['grain', 8.748986474532711e-237], ['corn', 5.160095468323234e-244], ['gas', 1.4098275290494474e-244]]
21542
['dmk', 'yen', 'dlr', 'money-fx'] [['money-fx', 1.0], ['dlr', 1.5545448503054055e-106], ['interest', 7.866782880468945e-166], ['trade', 1.1342357941045435e-246], ['yen', 3.617230094026259e-247], ['gnp', 1.0816515122788769e-248], ['acq', 2.3944928864641406e-277], ['ship', 2.17282595615502e-288], ['crude', 1.5461722173494344e-297], ['grain', 0.0]]
21555
['acq'] [['rubber', 0.8110475376296263], ['housing', 0.18888820520363336], ['ship', 5.175344298047375e-05], ['acq', 1.2500116891967357e-05], ['interest', 2.667977904736659e-09], ['money-fx', 6.165549970406476e-10], ['crude', 2.880060559601757e-10], ['grain', 1.842214655864592e-